In [ ]:
import dataset
import classificationModel
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import OrdinalEncoder
from torch.utils.data import random_split

In [ ]:
categories = ["chirping_birds", "crackling_fire", "hand_saw", "chainsaw", "helicopter"]
audio_path = './ESC-50/audio/'
meta_path = './ESC-50/meta/'
nothing_path = './room_sounds/'

data = pd.read_csv(meta_path + 'esc50.csv')

## Remove rows where the category does not belong to categories
data = data[data.category.isin(categories)]

re_encoder = OrdinalEncoder(dtype=np.long)
re_encoder.fit(data[["category"]])
data[["category"]] = re_encoder.transform(data[["category"]])

print(re_encoder.inverse_transform(np.array([0,1,2,3,4]).reshape(-1,1)))

In [ ]:
### Small test
myModel = classificationModel.AudioClassifier()
myModel.load_state_dict(torch.load('myModel.pt'))
categoryList = ['chainsaw','chirping_birds','crackling_fire','hand_saw','helicopter','nothing']
ds = dataset.SoundDS(data,audio_path,False)
myModel.eval()
classified_correctly = 0
for i in range(0,3890):
    sound = ds.getAudio(i)
    input, category = ds[i]
    input_m, input_s = input.mean(), input.std()
    input = (input - input_m) / input_s
    input = input.reshape((1,1,20,10))
    inputs = torch.empty((16,1,20,10))
    for i in range(0,16):
        inputs[i,:,:,:] = input
    # dataset.AudioUtil.playSound(sound)
    output = myModel(inputs)
    _, prediction = torch.max(output,1)
    if prediction[0] == category:
        print("classified correctly!")
        classified_correctly += 1
    print(prediction)
print(classified_correctly)